# Mission to Mars

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

### Initiate Browser

In [18]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Alex\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


## NASA Mars News

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

browser.visit(url)
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Retrieve the latest news title
news_title=soup.select_one('ul.item_list li.slide').find_all('div', class_='content_title')[0].text
# Retrieve the latest news paragraph
news_p=soup.find_all('div', class_='article_teaser_body')[0].text
print(news_title)
print(news_p)

Another First: Perseverance Captures the Sounds of Driving on Mars
NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.


## JPL Mars Space Images - Featured Image

In [5]:
jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(jpl_url)

In [6]:
full_image = browser.find_by_css('button.btn.btn-outline-light')
full_image.click()


In [8]:
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [9]:
# Relative path
img_url_rel = img_soup.select_one('div.fancybox-inner img').get("src")
img_url_rel

'image/featured/mars1.jpg'

In [10]:
# Absolute path
featured_image_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'
# browser.visit(featured_image_url)
print(featured_image_url)


https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


## Mars Facts

In [11]:
# Scrape Mars facts from https://space-facts.com/mars/
facts_url='https://space-facts.com/mars/'
tables=pd.read_html(facts_url)
facts = tables[0]
facts = facts.rename(columns={0:'Fact',1:'Measurement'})
facts

,Fact,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
fact_html = facts.to_html()
fact_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Fact</th>\n      <th>Measurement</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium B

## Mars Hemispheres

In [19]:
#Visit USGS website to grab the four hemisphere image urls along with their titles
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url)

In [21]:
#Create empty list for urls, use splinter to grab urls
hemisphere_image_urls = []
links = browser.find_by_css('a.product-item h3')

#Iterate through each hemisphere data
for i in range(len(links)):
    hemi_dict = {}
    browser.find_by_css('a.product-item h3')[i].click()
    hemi_dict['title'] = browser.find_by_css('h2.title').text
    hemi_img = browser.find_by_text('Sample')
    hemi_img.click()
    hemi_dict['img_url'] = hemi_img['href']
    hemisphere_image_urls.append(hemi_dict)
    browser.back()

hemisphere_image_urls    

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [16]:
browser.quit()